In [1]:
from pymongo import MongoClient
import pandas as pd
import json as js
import folium

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [54]:
rest = list(coll.find({ "$and": [ {"number_of_employees":{"$gt":100} }, {"founded_year":{"$gt":2000}},{"funding_rounds.raised_amount":{"$gt":1000000}}] } ))

In [55]:
type(rest)

list

In [49]:
'''
def getLocation(company):
    location_empty = []
    for i in range(len(company["offices"])):
        longitude = company['offices'][i]['longitude']
        latitude = company['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location_empty.append(loc)
    return location_empty
'''

In [92]:
# there is a problem with offices. many companies have many. with this loop you'll get all
location_list=[]
for x in rest:
    for y in x["offices"]:
        #print(y)
        #print(y["longitude"])
        longitude=y["longitude"]
        latitude=y["latitude"]
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location_list.append(loc)    

-122.151801
-6.267494
-73.9792469
-122.4169244
-77.245195
-73.996876
-118.380954
None
None
-83.167884
-118.1327468
None
None
None
None
-111.938739
-122.0899512
-73.995625
-122.07948
None
None
None
-122.402195
None
None
None
None
-122.206893
-122.39071
-122.079017
None
None
None
None
-122.073635
-122.143701
-122.325248
-122.178557
-80.004195
-121.8731805
-77.3592746
-0.1394759
-97.717667
-122.4186
-121.935688
-71.086596
None
None
None
None
None
None
4.5464658
None
-122.0727749
None
-122.4084994
None
-73.971655
None
None
None
18.0580765
None
None
None
-79.5983626
None
-121.948399
-122.406676
-72.188584
-79.338246
None
None
None
-122.286493
-94.597896
-73.992463
-122.401406
-81.879069
-79.3953743
-0.1229212
11.5582609
2.3797825
121.4770384
-78.815635
-122.084171
None
None
None
-118.456806
None
-122.156579
None
None
None
-73.9912208
None
None
-122.347533
-122.3293984
-122.285714
-122.40414
-122.394354
-122.3331466
-122.421344
-71.52679
None
-95.712891
-73.960717
None
None
-122.158804
-122.

{'description': 'Headquarters', 'address1': '1601 Willow Road', 'address2': '', 'zip_code': '94025', 'city': 'Menlo Park', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.41605, 'longitude': -122.151801}
{'description': 'Europe HQ', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'Dublin', 'state_code': None, 'country_code': 'IRL', 'latitude': 53.344104, 'longitude': -6.267494}
{'description': 'New York', 'address1': '340 Madison Ave', 'address2': '', 'zip_code': '10017', 'city': 'New York', 'state_code': 'NY', 'country_code': 'USA', 'latitude': 40.7557162, 'longitude': -73.9792469}
{'description': '', 'address1': '1355 Market St.', 'address2': '', 'zip_code': '94103', 'city': 'San Francisco', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.7768052, 'longitude': -122.4169244}
{'description': 'HQ - Virginia', 'address1': '1595 Spring Hill Road', 'address2': 'Suite 300', 'zip_code': '22182', 'city': 'Vienna', 'state_code': 'VA', 'country_code': 'USA', 'latitud

In [45]:
def getLocation(company):
    location_empty=[]
    

In [46]:
getLocation(rest)

AttributeError: 'list' object has no attribute 'x'

In [50]:
#si quiero añadir información a la colección $set (para el geoindex)
for place in rest:
    value = {"$set": {'location':getLocation(place)}}
    coll.update_one(place,value)

In [51]:
# eliminar las antiguas coordenadas
for place in rest:
    value = {"$unset": {'offices.latitude':''}}
    value = {"$unset": {'offices.longitude':''}}
    coll.update_one(place,value)
    # <collection>.update_one(<document>,<value>)

In [54]:
'''
prefered_city = coll.find(
    {"location":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-73.9712,40.7831]
       },
       "$maxDistance":10000
      }
     }
    }
)
prefered_city=list(prefered_city)
'''

'\nprefered_city = coll.find(\n    {"location":\n     {"$near":\n      {"$geometry":\n       {"type":"Point",\n        "coordinates":[-73.9712,40.7831]\n       },\n       "$maxDistance":10000\n      }\n     }\n    }\n)\nprefered_city=list(prefered_city)\n'

In [58]:
map_city=folium.Map(location=[40.7221,-73.9712], zoom_start=12)
map_city
'''
for wendy in manhattanWendys:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)
    
for wendy in manhattanWendys[:5]:
    folium.Marker(wendy['location']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='blue'), 
                   ).add_to(map_city)
'''

"\nfor wendy in manhattanWendys:\n    folium.Marker(wendy['location']['coordinates'][::-1],\n                    radius=2,\n                    icon=folium.Icon(icon='cloud',color='red'), \n                   ).add_to(map_city)\n    \nfor wendy in manhattanWendys[:5]:\n    folium.Marker(wendy['location']['coordinates'][::-1],\n                    radius=2,\n                    icon=folium.Icon(icon='cloud',color='blue'), \n                   ).add_to(map_city)\n"